# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

In [1]:
# importeren libs
import numpy as np
import pandas as pd
from itertools import product
import pickle
import copy

import Importeer 
import Hulpfuncties
import CrewCombined
import Dataframe
import Controleren
import InplanFuncties
from Class import MogelijkeCombinaties, Rooster

Het gegeven excelbestand wat de informatie vat, is op een aantal punten aangepast voordat het is geimporteerd. 
Het bestand heeft voor iedere skill een aparte kolom. Mocht de skill nodig zijn voor de gegeven opdracht, dan staat er een 'x' in deze kolom. Ditzelfde is gedaan met de skills van medewerkers, de dagen waarop de medewerkers werken en voor de kamers waarin de opdrachten moeten worden uitgevoerd. Voor de additional tasks zijn er een aantal opdracht die in iedere sprint moeten worden uitgevoerd. Deze zijn opgesplitst, voor iedere sprint een aparte lijn. Hierbij is ook een kolom toegevoegd met het aantal frequenties voor deze opdracht. Ook is er een kolom toegevoegd bij het tabblad 'additional tasks' waarin staat aangegeven met hoeveel personen het minimaal moet worden uitgevoerd.
Hieronder worden de verschillende tabbladen ingelezen en toegekent aan een dataframe.
Verdere bewerkingen op de data worden gedaan in dit document.


In [2]:
# CONSTANTE
SPRINT = 'S2' # welke sprint
# mensen indelen voor de standaard taken, met 0 als eerste crewlid en zo verder
OSHA = [4,9,0] 
crewDirecter = 6
science = 9
PR = 0
legal = 4
prepCheck = 5
prepTrouble = 12

In [3]:
# Importeren van verschillende sheets. Dit wordt gedaan met functies.
# in deze functies wordt het ook in het goede format gezet.
Importeer.AdditionalTasks()
dfCrew = Importeer.Crew()
dfRooms = Importeer.Rooms()
dfSprint = Importeer.dfSprint(SPRINT)

In [4]:
# CrewCombined.MakeCrewCombined(dfCrew) # voor het maken van dfCrewcombined.csv (duurt even)
dfCrewCombined = pd.read_csv('CrewCombined.csv') # lezen van csv zodra die in wdir staat.

**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken, daarna een half uur pauze
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' kan voor ieder crewlid worden gedaan, geen beperking in aantal 
- crewleden moet te samen aan de skills komen voor de opdracht.
- Mensen die skill één niet hebben, hoeven deze niet te leren
- Een crewleden kunnen alleen aan eengesloten uren aan een project task werken. Dit mag niet worden doorbroken met een andere task. Met additional tasks mag dit wel. 
- iedereen moet tegelijk aan het project werken als ze op hetzelfde project staan.

Moet A volledig worden afgerond of maar 1 keer van het aantal frequenties? Moet volledig worden ingepland

We maken een soort kalender, startend op 6 januari 2020 en geen weekend dagen erin.
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- een kolom met dataframes met de volgende punten:
    - tijd per halfuur
    - persoon
    - aantal uren te gaan
    - hierin staan de projecten met de kamer (is dit zo?)
- Of het een werkdag is (T/F)
- In welke sprint de dag zich bevindt.

In [5]:
dfKalenderCrew = Dataframe.LegeKalender() # het maken van een lege kalender voor de crew

In [6]:
dfKalenderRoom = pickle.loads(pickle.dumps(dfKalenderCrew)) # het maken van een kalender van de kamers, tot dit punt waren de df's hetzelfde

In [7]:
## het toevoegen van de mogelijkheden bij een project met kamer en de combinatie van crew 
dfSprint.iloc[:,26]= dfSprint.apply(lambda rij : Hulpfuncties.ToevoegenMogelijkeCombinaties(rij, 
                                                    dfSprint, dfCrewCombined), axis = 1)

# niet helemaal de frequency, maar meer een indicatie hoe lastig de opdracht is in te plannen
dfSprint.iloc[:,27] = dfSprint['Frequency']*dfSprint['MinReqCrew']*dfSprint['Duration']

In [8]:
# het schrijven van dfSprint naar een pickle bestand zodat deze makkelijk kan worden geladen
filehandler = open('dfSprint2_begin.obj', 'wb') 
pickle.dump(dfSprint, filehandler)

In [9]:
# het lezen van dfSprint, met meerdere keren runnen een heel stuk sneller
filehandler = open('dfSprint2_begin.obj', 'rb') 
dfSprint = pickle.load(filehandler)

In [10]:
# de standaard roosters voor een dag voor de kamer en de crew
StandaardRooster = Dataframe.RoosterCrew(dfCrew)
dfRoosterKamer = Dataframe.RoosterKamer(dfRooms)

In [11]:
dfCrew['Uren']=dfCrew.apply(lambda row: (row.mon + row.tue + row.wed + row.thu + row.fri)*70, axis=1) # toevoegen van het aantal uren dat nog gemaakt kan worden door de crew in een sprint

In [12]:
# inplannen stand-up op dinsdagochtend
dfRoosterDinsdag = copy.deepcopy(StandaardRooster)
dfRoosterDinsdag.iloc[6:15,1:14] = "weekly stand-up" 
dfRoosterDinsdag.iloc[0,1:14] = 2.5 # uren te gaan voor iedereen op die dag

dfCrew['Uren'] -= 45 # totaal aantal uren te gaan in een sprint aanpassen

dfRoosterDinsdag.iloc[1:6,[x+1 for x in OSHA]] = "OSHA" # osha inplannen en aanpassen van de uren op de dag, het totaal wordt aangepast in de functie inplannenosha
dfRoosterDinsdag.iloc[0,[x+1 for x in OSHA]] = 0

dfRoosterKamerDinsdag = copy.deepcopy(dfRoosterKamer)
dfRoosterKamerDinsdag.loc[6:15, 'workshop A'] ="weekly stand-up" # een kamer voor stand-up inroosteren

In [13]:
for i in range(0,5): # het maken van de lege roosters voor week 1
    if i== 1:
        rooster = copy.deepcopy(dfRoosterDinsdag)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamerDinsdag))
    else:
        rooster = copy.deepcopy(StandaardRooster)
        rooster.iloc[0,1:] = (np.array(rooster.iloc[0,1:])* np.array(dfCrew.iloc[:,14+i])).tolist() # het maken van de lijst urentegaan met de vermenigvuldiging van de lijst met dagen dat iedereen werkt en een lijst met alleen maar 7's, resultaat: 7 als iemand werkt, als iemand niet werkt, 0.
        dfKalenderCrew.DagRooster[i] = Rooster(rooster)
        dfKalenderRoom.DagRooster[i] = Rooster(copy.deepcopy(dfRoosterKamer))
    
for i in range(1,10): # het kopieren van de roosters van week 1 voor weken in de sprint
    for j in range(0,5):
        dfKalenderCrew.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderCrew.DagRooster[j])
        dfKalenderRoom.DagRooster[i*5 + j] = copy.deepcopy(dfKalenderRoom.DagRooster[j])

C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Robbert\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [14]:
dfSprintAddprep, dfSprintAddpost = Importeer.AdditionalTasks(SPRINT) # importeren van de perp en post, niet van andere add tasks

!TODO uit dfSprintAdd de check-up 1.1 of 1.2 halen, trouble shooting 

In [15]:
dfSprintAdd = Importeer.SprintAdditional(SPRINT) # binnenhalen van de add tasks, niet de prep en post

In [16]:
dfKalenderCrew, dfSprintAdd = InplanFuncties.InplannenOSHA(5, dfKalenderCrew, dfSprintAdd, OSHA, dfCrew) # inplannen van osha op woensdag voor 5 uur.

C:\Users\Robbert\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
# reserveren uren director etc.

dfCrew.Uren.iloc[crewDirecter] -= dfSprintAdd[dfSprintAdd.Task == "crew director"]["DurationExp.1"].tolist()[0]
dfCrew.Uren.iloc[crewDirecter] -= (dfSprintAdd[dfSprintAdd.Task == "meetup with stakeholders"]["DurationExp.1"].tolist()[0])
# Werkt, alleen pakt hij standaard de verkeerd (met 12+3*3)

dfCrew.Uren.iloc[science] -= (dfSprintAdd[dfSprintAdd.Task == "scientific advisor"]["DurationExp.1"].tolist()[0])
                              

dfCrew.Uren.iloc[PR] -= (dfSprintAdd[dfSprintAdd.Task == "PR"]["DurationExp.1"].tolist()[0])
dfCrew.Uren.iloc[legal] -= (dfSprintAdd[dfSprintAdd.Task == "legal assistant"]["DurationExp.1"].tolist()[0])
dfCrew.Uren.iloc[prepCheck] -= (dfSprintAdd[dfSprintAdd.Task == "prep check-up"]["DurationExp.1"].tolist()[0])
dfCrew.Uren.iloc[prepTrouble] -= (dfSprintAdd[dfSprintAdd.Task == "prep troubleshooting"]["DurationExp.1"].tolist()[0])


In [18]:
def InplannenAdd(taskdf, dfKalenderCrew, dfKalenderRoom, dfSprint, dfAdd, dfCrew, prep = True, inplannen = False): 
#     In deze functie wordt gekeken of een prep of post kan worden ingepland. 
#     Mocht inplannen op True staan wordt deze ingepland zodra het kan.
#     In de functie wordt een copy gemaakt van dfKalenderCrew, dfKalenderRoom, dfAdd en dfCrew. 
#     De andere dataframes die worden meegegeven worden niet aangepast, maar alleen gebruikt.

    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempAdd = pickle.loads(pickle.dumps(dfAdd))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
    
    gelukt = False

    dagIndex, weken, maxDagIndex = [0,[1],5] if prep else [40,[9,10],50] # constante voor prep of post
    
    if (taskdf.AdditionalReq == 0):
        
        task = taskdf.Task[5:] # de naam van de task opslaan
        crewlijst = dfSprint[dfSprint.Task == task].iloc[0,:].Crew # de crew vinden die het project doen
        
        minCrew = taskdf['People.1'] # we plannen de opdrachten eigenlijk altijd in met de min crew, hier wordt deze gezet.

        crewlijst = Hulpfuncties.somOverigeUren(crewlijst, weken, dfKalenderCrew) # kijken hoeveel uren ieder nog over heeft.
        crewList = crewlijst['crew'].tolist()

        for crew in crewList: # checken voor het totaal aantal uren en extra crew. 
            # als er te weinig uren zijn of het een extra crewlid is, dan wordt het lid verwijderd.
            if ((dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"]) | (crew >12)):
                crewlijst = crewlijst[crewlijst.crew != crew]
        if (len(crewlijst) < minCrew): # mocht er niet voldoende crew zijn, dan returnt de functie de oude staat.
            return [False, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]
        
        crewlijst = crewlijst['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew)) # de lijst 6 lang maken met toevoegen van 20
       
    elif ((len(taskdf.AdditionalReq) > 5) & (len(taskdf.AdditionalReq) < 8)):
        try:
            crewlijst = dfSprint[dfSprint.Task == taskdf.AdditionalReq].iloc[0,:].Crew
        except:
            filehandler = open('dfSprint1_eind.obj', 'rb') # Moet worden verandert met de sprint
            dfSprintVorig = pickle.load(filehandler)
            crewlijst = dfSprintVorig[dfSprintVorig.Task == taskdf.AdditionalReq].iloc[0,:].Crew
            
        crewlijst = Hulpfuncties.somOverigeUren(crewlijst, weken, dfKalenderCrew) # kijken hoeveel uren ieder nog over heeft.
        crewList = crewlijst['crew'].tolist()
        
        for crew in crewList: # checken voor het totaal aantal uren en extra crew. 
            # als er te weinig uren zijn of het een extra crewlid is, dan wordt het lid verwijderd.
            if ((dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"]) | (crew >12)):
                crewlijst = crewlijst[crewlijst.crew != crew]
        if (len(crewlijst) < len(crewList)): # mocht het aantal mensen niet gelijk aan elkaar zijn, 
                                        # kan het niet door deze crew worden gedaan want iedereen die het project doet
                                        # moet met deze eis ook de prep en of post doen.
            return [False, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]
        
        crewlijst = crewlijst['crew'].to_list()
        crewlijst.extend([20]*(6-len(crewlijst))) # de lijst 6 lang maken met toevoegen van 20   
        
    elif (len(taskdf.AdditionalReq) > 8 ): # bij deze moet één persoon de task uitvoeren die zowel bij A of B zit
        taskA = taskdf.AdditionalReq[0:7]
        taskB = taskdf.AdditionalReq[-7:]
        taskACrew = dfSprint[dfSprint.Task == taskA].iloc[0,:].Crew
        taskBCrew = dfSprint[dfSprint.Task == taskB].iloc[0,:].Crew # vinden van de crew van task A en B
        b = list(set(taskACrew)&set(taskBCrew))
        try:
            b.remove(20)
        except:
            None
        
        minCrew = taskdf['People.1'] # we plannen de opdrachten eigenlijk altijd in met de min crew, hier wordt deze gezet.
        crewlijst = Hulpfuncties.somOverigeUren(b, weken, dfKalenderCrew) # kijken hoeveel uren ieder nog over heeft.
        crewList = crewlijst['crew'].tolist()
        
        for crew in crewList: # checken voor het totaal aantal uren en extra crew. 
            # als er te weinig uren zijn of het een extra crewlid is, dan wordt het lid verwijderd.
            if ((dfCrew['Uren'].iloc[crew] < taskdf["DurationExp.1"]) | (crew >12)):
                crewlijst = crewlijst[crewlijst.crew != crew]
        if (len(crewlijst) < minCrew): # mocht er niet voldoende crew zijn, dan returnt de functie de oude staat.
            return [False, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]
        
        crewlijst = crewlijst['crew'].iloc[:minCrew].to_list()
        crewlijst.extend([20]*(6-minCrew)) # de lijst 6 lang maken met toevoegen van 20
        
    # het toekennen van een aantal constanten
    kolom = 1 # voor het nummer van de kolom voor de duur en het aantal personen
    
    lijstVrijeUren = [0]*16
    controle = True
    dagIndexKopie = dagIndex
    halfuur = 0
    
    if (int(round(taskdf[str("DurationExp." + str(kolom))]*2)) == 0): #mocht de duration van een task nul zijn, dan wordt gelukt op True gezet.
        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True
        gelukt = True
        print("duration is nul dus vroegtijdig afgebroken.")
        if inplannen:
            return [True, dfKalenderCrew, dfKalenderRoom, dfAdd, dfCrew]
        else:
            return [True, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]
        
    else:
        while halfuur < int(round(taskdf[str("DurationExp." + str(kolom))]*2)): # hier wordt per half uur een task ingepland tot de duration keer twee is bereikt (dan is het halve uren)
            for i in crewlijst:
                if (i == 20): # breakt bij 20 want er is geen crew met nummer 20, maar staat wel in de lijst zodat iedere lijst even lang is.
                    break
                if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(0.5)): # als een crew geen uren meer heeft op een dag
                    controle = False
                    break 
                # lijst maken met 1'en en 0'en waarbij 1 betekent dat een crew al iets heeft te doen en 0 dat er nog niks gepland is.
                lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) # voor iedere op hetzelfde moment

            if ((Hulpfuncties.contains([0], lijstVrijeUren) != False) & controle): # als er ergens een halfuur vrij is en iedereen minimaal een halfuur heeft
                rijbegin , rijeind = Hulpfuncties.contains([0], lijstVrijeUren) # het vinden van het moment waar de opdracht kan worden ingepland

                kamerGevonden = False # initialiseren variabele

                kamers = Hulpfuncties.ZoekKamers(taskdf.iloc[14:20], dfAdd) # mogelijke kamers zoeken

                for kamer in kamers: # itereren over de kamers
                    if ([0] == dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1].tolist()):

                        if inplannen: # inplannen als dit is meegegeven
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[kamer].iloc[rijbegin:rijeind+1] = taskdf.Task
                        kamerGevonden = True
                        controle = True
                        break
                    elif ((kamer == kamers[-1]) & (kamerGevonden == False)): # geen kamer gevonden of kamerGevonden nog steeds valse                     
                        controle = False

                if (controle == True):
                    for i in crewlijst:
                        if i == 20:
                            break
                        
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task # het plaatsen van de task in het rooster van de crew
                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(0.5) # aanpassen van de uren te gaan voor een crewlid

                    halfuur +=1 # verhogen van het aantal halve uren 

                    if (halfuur == (int(round(taskdf[str("DurationExp." + str(kolom))]*2)))): # zodra het aantal halve uren gelijk is aan het totaal aantal halve uren
                        geluktCrew, dfCrew = Hulpfuncties.CheckCrewUren(crewlijst, dfCrew, (int(round(taskdf[str("DurationExp." + str(kolom))])))) # het aanpassen van het aantal uren van de crew over de sprint 
                        if not geluktCrew:
                            print("er is iets fout gegaan")
                        dfAdd.at[dfAdd.index[dfAdd['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst # veranderen van de crewlijst in het dataframe van de additional tasks
                        dfAdd.loc[dfAdd['Task'] == taskdf.Task, 'Voltooid'] = True # de task op voltooid zetten
                        print(str(taskdf.Task) + "is daadwerkelijk ingepland")

                        if inplannen: # inplannen zodra deze True is en de aangepaste teruggeven, ander de kopien terugzetten
                            return [True, dfKalenderCrew, dfKalenderRoom, dfAdd, dfCrew]
                        else:
                            return [True, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew]

            else: # moment niet gevonden op de dag, door naar de volgende dag, hierbij worden een aantal variabele gereset

                dagIndexKopie+=1

                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == maxDagIndex):
                    break

    print(str(taskdf.Task) + " Niet ingepland of: " + str(gelukt))
    return [gelukt, dfTempCrew, dfTempRoom, dfTempAdd, dfCopyCrew] # mocht de functie hierkomen, dan worden de kopien teruggegeven.

In [19]:
def InplannenOpdracht(taskdf, dagIndex, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfAddPost, dfSprint, dfCrew, crewA = []): 

# task moet een rij van het dataframe dfSprint zijn en een dag (als index) van een rij in dfKalenderCrew 
# verschil ten opzichten van 2020-1-6)
# CrewA wordt meegegeven zodra de task een eis heeft, in het geval mensen van task x 
# ook bij deze task moeten worden gebruikt.
     
    # kopien maken van de meegegeven dataframes
    dfTempCrew = pickle.loads(pickle.dumps(dfKalenderCrew))
    dfTempRoom = pickle.loads(pickle.dumps(dfKalenderRoom))
    dfTempSprint = pickle.loads(pickle.dumps(dfSprint))
    dfTempPrep = pickle.loads(pickle.dumps(dfAddPrep))
    dfTempPost = pickle.loads(pickle.dumps(dfAddPost))
    dfCopyCrew = pickle.loads(pickle.dumps(dfCrew))
    
    for crewlijst in taskdf.Mogelijkheden.dfMogelijkheden.iloc[:,1].tolist(): # itereren over de mogelijkheden van crewcombinaties
        checkUren, dfCrew = Hulpfuncties.CheckCrewUren(crewlijst, dfCrew, taskdf.Duration * taskdf.Frequency * 7) # kijken of het kan met de uren van de crew, als het kan worden deze ook gelijk aangepast.
        
        if (checkUren):
            a = list(set(crewlijst))
            try:
                a.remove(20) # 20 verwijderen zodat er alleen een lijst overblijft met de crewleden.
            except:
                None

            # if ((len(set(a) & set(crewA)) >= 1) | (len(crewA)==0)): # de overlap tussen de crewlijst en crewA moet groter zijn dan 1 als crewA ongelijk is aan nul
            # aantal variabele worden hier geinitialiseerd.
            lijstVrijeUren = [0]*16
            lijstVrijeUrenMorgen = [0]*16
            controle = True
            urenOver = 7 
            urenOverMorgen = 7 
            dagIndexKopie = dagIndex
            freq = 1
            urenOphogen = False
            geluktPost = False


            while (freq <= int(taskdf.Frequency)) & (taskdf.Voltooid == False): # zolang de frequentie niet de maximale frequentie is en de task nog niet voltooid is.
                urenOver = 7 
                urenOverMorgen = 7 # voor het aantal uren te gaan 

                if (taskdf.Eis != 'no') & (dagIndex == dagIndexKopie): # voor als er een eis is, dit hoeft alleen de eerste keer, vandaar de dag==dagkopie
                    eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
                    lijstVrijeUren = [1]*int(eis["Moment Voltooid"]) + [0]*(16-int(eis["Moment Voltooid"])) # wanneer de eis is voltooid
                if urenOphogen:
                    rijbegin += 1
                    lijstVrijeUren = [1]*rijbegin + [0]*(16-rijbegin) 
                    urenOphogen = False
                for i in crewlijst:
                    if (i == 20):
                        break

                    urenOver = min(float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]), urenOver) # minimum van alle uren te gaan in crewlijst
                    urenOverMorgen = min(float(dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1]), urenOverMorgen) # minimum voor de dag erna

                    if ((float(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1]))< float(taskdf.Duration)): # uren te gaan kleiner dan de duration 
                        controle = False

                        break
                    # Het combineren van de dagen. zodra iemand niet kan op een moment, kan de hele task niet op dat moment
                    lijstVrijeUren = Hulpfuncties.CombinatieLijst(dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[1:,i+1].tolist(), lijstVrijeUren) 
                duration = [0]*int(round(taskdf.Duration*2)) # lijst maken met nullen in de lengte van het aantal halve uren

                if (Hulpfuncties.contains(duration, lijstVrijeUren) != False) & controle: # als de duration in de lijstvrije uren past
                    rijbegin , rijeind = Hulpfuncties.contains(duration, lijstVrijeUren)

                    kamerGevonden = False
                    for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]: # itereren over de kamer mogelijkheden
                        if (duration == dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1].tolist()): # als de duration gelijk is aan het rooster van de kamer (leeg is) 
                            dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[rijbegin:rijeind+1] = taskdf.Task + " freq: " + str(freq) # inplannen van een opdracht in een kamer
                            kamer = i
                            kamerGevonden = True
                            controle = True
                            break
                        elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1]) & (kamerGevonden == False)):
                            controle = False # hierdoor hoeft alleen controle te worden gebruikt en niet meer kamergevonden
                            urenOphogen = True
                    if (controle == True):
                        for i in crewlijst:
                            if i == 20:
                                break

                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[rijbegin+1:rijeind+2,i+1] = taskdf.Task + " freq: " + str(freq) + " " + str(kamer) # invullen van task, freq en kamer
                            dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - float(taskdf.Duration) # aanpassen van het aantal uren te gaan.

                        if (freq == int(taskdf.Frequency)): # Bij de laatste frequentie
                            # vinden van prep en post
                            dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]
                            dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]

                            dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst # inplannen van crew zodat deze kunnen worden gebruikt in de functies hieronder.

                            geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                      prep = True) # testen of prep kan worden ingepland

                            geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                      prep = False) # testen of post kan worden ingepland

                            if (geluktPost & geluktPrep): # zodra de prep en post kunnen worden ingepland, deze inplannen

                                geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom,dfCrew=dfCrew, 
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                        prep = True, inplannen=True)
                                geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                      dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                      dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                      prep = False, inplannen=True)

                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = rijeind +1
                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie
                                dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True


                                print(str(taskdf.Task) + " is ingepland, met prep en post")
                                return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost, dfCrew]
                            geluktPost = False
#                             print(str(taskdf.Task) + " freq: " + str(freq) + " crew: " +str(crewlijst) + "task voltooid: "+ str(taskdf.Voltooid) + " " + str(dagIndexKopie))    
                        freq += 1 # ophogen van de frequentie

                else: # het inplannen over meerdere dagen
                    if ((urenOver + urenOverMorgen >= taskdf.Duration) & (urenOver>0) & (dagIndexKopie != 9)): # controleren of mensen uren over hebben en of het niet de laatste dag van de week is.

                        lijstVrijeEindUren = [0] * int(urenOver *2) # lijst van nullen met de lengt het aantal halve uren van de uren die op de eerste dag worden gemaakt

                        for i in crewlijst:
                            if (i == 20):
                                break
                            eindUrenLijst = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[17- int(urenOver*2):,i+1].tolist()
                            lijstVrijeEindUren = Hulpfuncties.CombinatieLijst(eindUrenLijst,lijstVrijeEindUren) # combineren van de lijsten om later te kijken wanneer de task kan worden ingepland

                        startMomentLijst = Hulpfuncties.get_last_non_zero_index(lijstVrijeEindUren,0) # wanneer kan er worden gestart met de opdracht
                        startMomentLijst = startMomentLijst + 1 if lijstVrijeEindUren[0] != 0 else startMomentLijst # zodra de lijsteind [1,0,0] is, moet er + 1 worden gedaan, anders niet
                        mogelijkeDurationEersteDag = len(lijstVrijeEindUren[startMomentLijst:]) # dit kan langer zijn dan de uren over
                        durationEersteDag = min(int(urenOver *2), mogelijkeDurationEersteDag) # , taskask.Duration*2 # de maximale lengte bepalen in halve uren

                        if ((durationEersteDag>0) & (taskdf.Duration*2 > durationEersteDag)):
                            overigeDuration = taskdf.Duration*2 -durationEersteDag # duur voor de volgende dag, rest duur
                            lijstVrijeBeginUren = [0]* int(overigeDuration)
                            for i in crewlijst:
                                if (i == 20):
                                    break
                                beginUrenLijst = dfKalenderCrew.iloc[dagIndexKopie + 1,:].DagRooster.dfRooster.iloc[1:int(overigeDuration)+1,i+1].tolist()
                                try: # dit ging mis bij de tweede iteratie
                                    lijstVrijeBeginUren = Hulpfuncties.CombinatieLijst(beginUrenLijst,lijstVrijeBeginUren)
                                except: # hier meer shit printen
# 2-12 
                                    print("Er gaat nu iets fout met de lengte van een van de lijsten")
                                    print("begin: " + str(beginUrenLijst) + " vrije: " + str(lijstVrijeBeginUren) + " " + str(taskdf.Task) + " "+ str(freq) + " " + str(crewlijst) + " overige duration: " + str(overigeDuration)) 
                                    return [dfTempCrew, dfTempRoom, dfTempSprint, dfTempPrep, dfTempPost, dfCopyCrew]

#                                 print("beginlijst: "+ str(lijstVrijeBeginUren) + "duration: " + str(overigeDuration) + " " + str(lijstVrijeEindUren)+ " " + str(mogelijkeDurationEersteDag)+ " " + str(lijstVrijeUren))

                            pastMorgen = Hulpfuncties.past_task_begin_dag(lijstVrijeBeginUren, overigeDuration)

#                                 print("test past: " +str(pastMorgen))
                            if ((pastMorgen) & (Hulpfuncties.extraCheckUrenMorgen(dagIndexKopie, crewlijst, dfKalenderCrew))): 
                                #vanaf hier wordt er daadwerkelijk ingepland voor over de dagen heen.
                                kamerGevonden = False

                                for i in taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0]: # itereren over de kamers

                                    if ((16 - Hulpfuncties.get_last_non_zero_index(dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[1:].tolist()) >= mogelijkeDurationEersteDag) &
                                        (Hulpfuncties.past_task_begin_dag(dfKalenderRoom.iloc[dagIndexKopie+1,:].DagRooster.dfRooster[i].iloc[1:].tolist(),overigeDuration))): # als de task past, wordt deze ingepland met een kamer
                                        dfKalenderRoom.iloc[dagIndexKopie,:].DagRooster.dfRooster[i].iloc[16-mogelijkeDurationEersteDag:] = taskdf.Task + " freq: " + str(freq)
                                        dfKalenderRoom.iloc[dagIndexKopie+1,:].DagRooster.dfRooster[i].iloc[0:int(overigeDuration)+1] = taskdf.Task + " freq: " + str(freq)
                                        kamer = i
                                        kamerGevonden = True
                                        controle = True
                                        break
                                    elif ((i == taskdf.Mogelijkheden.dfMogelijkheden.iloc[0,0][-1])& (kamerGevonden == False)):
                                        controle = False

                                # kamer gevonden, momenten gevonden zodra controle True is
                                if (controle == True):

                                    for i in crewlijst:
                                        if i == 20:
                                            break
                                        # inplannen voor alle crew op beide dagen en de uren aanpassen.
                                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[16-mogelijkeDurationEersteDag+1:,i+1] = taskdf.Task + " freq: " + str(freq) + " " + str(kamer) 
                                        dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[1:int(overigeDuration)+1,i+1] = taskdf.Task + " freq: " + str(freq) + " " + str(kamer)
                                        dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie,:].DagRooster.dfRooster.iloc[0,i+1] - (durationEersteDag/2)
                                        dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1] = dfKalenderCrew.iloc[dagIndexKopie+1,:].DagRooster.dfRooster.iloc[0,i+1] - (overigeDuration/2)

                                    if (freq == int(taskdf.Frequency)): # bij de laatste freq
                                        dfPrep = pd.DataFrame(dfAddPrep[dfAddPrep['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:] # vinden van prep en post
                                        dfPost = pd.DataFrame(dfAddPost[dfAddPost['Task'].str.contains(taskdf.Task)]).reset_index(drop=True).iloc[0,:]

                                        dfSprint.at[dfSprint.index[dfSprint['Task'] == taskdf.Task].tolist()[0], 'Crew'] = crewlijst # inplannen van crew voor de functies die volgen
                                        geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                                  prep = True) # testen of prep past
                                        
                                        if geluktPrep:
                                            geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                                      dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                      dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                                      prep = False)# testen of post past

                                        if (geluktPost & geluktPrep): # als het allebei kan, inplannen

                                            geluktPrep, dfKalenderCrew, dfKalenderRoom, dfAddPrep, dfCrew = InplannenAdd(taskdf=dfPrep, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom,dfCrew=dfCrew, 
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPrep, 
                                                                                                    prep = True, inplannen=True)
                                            geluktPost, dfKalenderCrew, dfKalenderRoom, dfAddPost, dfCrew = InplannenAdd(taskdf=dfPost, dfKalenderCrew=dfKalenderCrew,
                                                                                                  dfKalenderRoom=dfKalenderRoom, dfCrew=dfCrew,
                                                                                                  dfSprint=dfSprint, dfAdd=dfAddPost, 
                                                                                                  prep = False, inplannen=True)

                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Moment Voltooid'] = overigeDuration + 1
                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Dag Voltooid'] = dagIndexKopie + 1
                                            dfSprint.loc[dfSprint['Task'] == taskdf.Task, 'Voltooid'] = True


                                            print(str(taskdf.Task) + " is ingepland, met prep en post")
                                            return [dfKalenderCrew, dfKalenderRoom, dfSprint, dfAddPrep, dfAddPost, dfCrew]

                                        geluktPost = False
#                                         print(str(taskdf.Task) + " freq: " + str(freq) + " crew: " +str(crewlijst) + "task voltooid: "+ str(taskdf.Voltooid) + " " + str(dagIndexKopie))    
                                    freq += 1


                    dagIndexKopie+=1 
#                         print("dagIndex verhogen naar: " + str(dagIndexKopie))
                controle = True
                lijstVrijeUren = [0]*16
                if (dagIndexKopie == 10):
                    # met deze crew kan het niet worden ingepland dus weer opnieuw beginnen, dataframes terugzetten
                    dfKalenderCrew = pickle.loads(pickle.dumps(dfTempCrew))
                    dfKalenderRoom = pickle.loads(pickle.dumps(dfTempRoom))
                    dfSprint = pickle.loads(pickle.dumps(dfTempSprint))
                    dfAddPrep = pickle.loads(pickle.dumps(dfTempPrep))
                    dfAddPost = pickle.loads(pickle.dumps(dfTempPost))
                    dfCrew = pickle.loads(pickle.dumps(dfCopyCrew))
                    break
    return [dfTempCrew, dfTempRoom, dfTempSprint, dfTempPrep, dfTempPost, dfCopyCrew] # als het programma hierkomt, dan kan het met geen enkele crew worden ingepland dus kopien teruggeven
    
    

In [20]:
#kopien maken voor in de loop van zometeen
dfk = pickle.loads(pickle.dumps(dfKalenderCrew))
dfr = pickle.loads(pickle.dumps(dfKalenderRoom))
dfs = pickle.loads(pickle.dumps(dfSprint))
dfprep = pickle.loads(pickle.dumps(dfSprintAddprep))
dfpost = pickle.loads(pickle.dumps(dfSprintAddpost))
dfc = pickle.loads(pickle.dumps(dfCrew))

In [21]:
del dfCrewCombined
del dfRoosterDinsdag
del StandaardRooster
del rooster
del filehandler
del dfRoosterKamerDinsdag

In [22]:
import warnings
warnings.filterwarnings("ignore")

iteratie = 0
while (dfSprint.Voltooid.sum()< len(dfSprint)) & (iteratie < 3): # stop zodra alles is ingepland of als het aantal iteraties zijn bereikt.

#     sorteren wordt gedaan op voltooid en aantalmogelijkheden. hiervoor worden nog wel de tasks gezet die een eis zijn.
#     verder worden er kopien terug gezet naar voor de iteratie. alleen de volgorde wordt aangepast.

    
    dfSprint = dfSprint.sort_values(['Counter', 'AantalMogelijkheden'], ascending= [False, False]).reset_index(drop=True)
    
    rows = dfSprint.Task.tolist()
    counter = dfSprint.Counter.tolist()
    
    dfKalenderCrew = pickle.loads(pickle.dumps(dfk))
    dfKalenderRoom = pickle.loads(pickle.dumps(dfr))
    dfSprintAddprep = pickle.loads(pickle.dumps(dfprep))
    dfSprintAddpost = pickle.loads(pickle.dumps(dfpost))
    dfCrew = pickle.loads(pickle.dumps(dfc))
    
    dfSprint = pickle.loads(pickle.dumps(dfs))
    dfSprint = dfSprint.set_index('Task')
    dfSprint = dfSprint.reindex(index= rows)
    dfSprint = dfSprint.reset_index()
    dfSprint.Counter = counter
    print("kopieen gemaakt, iteratie is: "+str(iteratie))
    
    for i in range(0,len(dfSprint)): # itereren over de opdrachten
        task = dfSprint.iloc[i,:]
        print(task.Task)
        dagIndex = 5 # omdat we maar een week inplannen en die vervolgens kopieren
        
        crewA = []
        if task.Eis != 'no': # als er een eis is
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:]
            if eis.Voltooid == False: 
                print("eis inplannen")
                dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost, dfCrew = InplannenOpdracht(taskdf=eis,
                                                                   dagIndex=dagIndex, dfSprint=dfSprint,
                                                                   dfKalenderCrew=dfKalenderCrew, dfCrew=dfCrew,
                                                                   dfKalenderRoom=dfKalenderRoom,
                                                                   dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost)
            eis = dfSprint[dfSprint.Task == task.Eis].iloc[0,:] # ophalen van crew en of de task is voltooid
            dagIndex = int(eis['Dag Voltooid'])
            crewA = eis.Crew
        dfKalenderCrew, dfKalenderRoom, dfSprint, dfSprintAddprep, dfSprintAddpost, dfCrew = InplannenOpdracht(taskdf=task,
                                                               dagIndex=dagIndex, crewA=crewA,
                                                               dfKalenderCrew=dfKalenderCrew, dfCrew=dfCrew,
                                                               dfKalenderRoom=dfKalenderRoom,
                                                               dfAddPrep=dfSprintAddprep, dfAddPost=dfSprintAddpost,
                                                               dfSprint=dfSprint)
    iteratie += 1
    print(dfSprint.iloc[:,[0,20,21,22,23,27,28,29,30,31]])
    dfSprint.Counter += (1-dfSprint.Voltooid)


kopieen gemaakt, iteratie is: 0
S2C1T5A
[10, 5, 20, 20, 20, 20]
prep S2C1T5Ais daadwerkelijk ingepland
[10, 5, 20, 20, 20, 20]
post S2C1T5Ais daadwerkelijk ingepland
[10, 5, 20, 20, 20, 20]
prep S2C1T5Ais daadwerkelijk ingepland
[10, 5, 20, 20, 20, 20]
post S2C1T5Ais daadwerkelijk ingepland
S2C1T5A is ingepland, met prep en post
S2C2T4A
[4, 20, 20, 20, 20, 20]
prep S2C2T4Ais daadwerkelijk ingepland
[4, 20, 20, 20, 20, 20]
post S2C2T4Ais daadwerkelijk ingepland
[4, 20, 20, 20, 20, 20]
prep S2C2T4Ais daadwerkelijk ingepland
[4, 20, 20, 20, 20, 20]
post S2C2T4Ais daadwerkelijk ingepland
S2C2T4A is ingepland, met prep en post
S2C3T2A
[11, 20, 20, 20, 20, 20]
prep S2C3T2Ais daadwerkelijk ingepland
[11, 20, 20, 20, 20, 20]
post S2C3T2Ais daadwerkelijk ingepland
[11, 20, 20, 20, 20, 20]
prep S2C3T2Ais daadwerkelijk ingepland
[11, 20, 20, 20, 20, 20]
post S2C3T2Ais daadwerkelijk ingepland
S2C3T2A is ingepland, met prep en post
S2C1T3
[8, 0, 20, 20, 20, 20]
prep S2C1T3is daadwerkelijk ingepland

!TODO Skill 8 (?) inplannen voor die mensen....

In [24]:
dfSprint.iloc[:,[0,20,21,22,23,25,27,28,29,30,31,32]]

,Task,MinReqCrew,MaxReqCrew,Frequency,Duration,Crew,AantalMogelijkheden,Counter,Voltooid,Eis,Moment Voltooid,Dag Voltooid
0,S2C1T5A,2.0,6.0,8.0,1.5,"[10, 5, 20, 20, 20, 20]",24.0,1,True,no,6,8
1,S2C2T4A,1.0,2.0,2.0,1.5,"[4, 20, 20, 20, 20, 20]",3.0,1,True,no,6,7
2,S2C3T2A,1.0,6.0,1.0,2.0,"[11, 20, 20, 20, 20, 20]",2.0,1,True,no,4,5
3,S2C1T3,2.0,6.0,8.0,2.0,"[8, 0, 20, 20, 20, 20]",32.0,0,True,no,6,9
4,S2C1T4,2.0,6.0,8.0,2.0,"[12, 9, 20, 20, 20, 20]",32.0,0,True,no,6,8
5,S2C1T2,2.0,6.0,4.0,2.0,"[6, 3, 20, 20, 20, 20]",16.0,0,True,no,3,6
6,S2C1T5B,2.0,6.0,4.0,2.0,"[11, 5, 20, 20, 20, 20]",16.0,0,True,S2C1T5A,8,9
7,S2C2T4B,2.0,6.0,4.0,2.0,"[8, 2, 20, 20, 20, 20]",16.0,0,True,S2C2T4A,14,9
8,S2C1T1,4.0,6.0,1.0,3.0,"[10, 7, 3, 1, 20, 20]",12.0,0,True,no,12,8
9,S2C3T1,3.0,4.0,1.0,3.0,"[11, 5, 4, 20, 20, 20]",9.0,0,True,no,12,7


In [25]:
dfCrew

,Crew members,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Skill 7,Skill 8,Skill 9,...,Skill 11,Skill 12,Skill 13,mon,tue,wed,thu,fri,Additional,Uren
0,Alyssa,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,1,1,0,39.0
1,Chris,1,0,0,0,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,18.0
2,Dennis,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,0,116.0
3,Iris,1,0,0,0,0,1,0,1,0,...,1,0,1,1,1,0,1,0,0,46.0
4,Jurgen,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,0,0,10.0
5,Lois,1,1,1,0,0,0,1,0,1,...,0,0,0,1,1,1,1,1,must have acquired skill 8 by the end of sprint 4,8.0
6,Melanie,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,6.0
7,Mitch,1,0,1,0,0,0,0,1,1,...,1,1,1,1,1,0,1,1,0,172.0
8,Rudy,1,1,0,0,0,0,1,1,1,...,1,0,0,1,1,1,1,1,0,81.0
9,Shane,1,0,0,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,36.5


In [26]:
dfSprintAddpost.iloc[:,[0,20,21,22,23,27,28,31,32,33,34]]

,Task,AdditionalReq,DurationExp.1,DurationStd.1,DurationExp.2,People.1,People.2,aantal keer,Sprint,Voltooid,Crew
0,post S2C1T1,S2C1T1,6.5,1.0,6.0,4,5,1.0,2.0,True,"[7, 10, 3, 1, 20, 20]"
1,post S2C1T2,0,32.0,6.0,19.5,1,2,1.0,2.0,True,"[6, 20, 20, 20, 20, 20]"
2,post S2C1T3,0,32.0,6.0,19.5,1,2,1.0,2.0,True,"[8, 20, 20, 20, 20, 20]"
3,post S2C1T4,0,32.0,6.0,19.5,1,2,1.0,2.0,True,"[12, 20, 20, 20, 20, 20]"
4,post S2C1T5A,0,32.0,6.0,19.5,1,2,1.0,2.0,True,"[5, 20, 20, 20, 20, 20]"
5,post S2C1T5B,0,4.5,0.5,4.5,1,1,1.0,2.0,True,"[11, 20, 20, 20, 20, 20]"
6,post S2C1T6,0,2.0,0.0,2.0,1,1,1.0,2.0,True,"[0, 20, 20, 20, 20, 20]"
7,post S2C2T1,S2C2T1,12.0,2.0,12.0,1,0,1.0,2.0,True,"[11, 7, 3, 1, 20, 20]"
8,post S2C2T2,0,11.0,2.0,6.5,1,2,1.0,2.0,True,"[11, 20, 20, 20, 20, 20]"
9,post S2C2T3,0,11.0,2.0,6.5,1,2,1.0,2.0,True,"[2, 20, 20, 20, 20, 20]"


In [27]:
dfSprintAddprep.iloc[:,[0,20,21,22,23,27,28,31,32,33,34]]

,Task,AdditionalReq,DurationExp.1,DurationStd.1,DurationExp.2,People.1,People.2,aantal keer,Sprint,Voltooid,Crew
0,prep S2C1T1,S2C1T1,9.5,1.5,8.5,4,5,1.0,2.0,True,"[7, 10, 3, 1, 20, 20]"
1,prep S2C1T2,0,6.5,1.0,6.5,1,1,1.0,2.0,True,"[6, 20, 20, 20, 20, 20]"
2,prep S2C1T3,0,6.5,1.0,6.5,1,1,1.0,2.0,True,"[8, 20, 20, 20, 20, 20]"
3,prep S2C1T4,0,6.5,1.0,6.5,1,1,1.0,2.0,True,"[12, 20, 20, 20, 20, 20]"
4,prep S2C1T5A,0,6.5,1.0,6.5,1,1,1.0,2.0,True,"[5, 20, 20, 20, 20, 20]"
5,prep S2C1T5B,0,0.0,0.0,0.0,1,1,1.0,2.0,True,20
6,prep S2C1T6,0,2.0,0.0,2.0,1,1,1.0,2.0,True,"[0, 20, 20, 20, 20, 20]"
7,prep S2C2T1,0,0.0,0.0,0.0,1,1,1.0,2.0,True,20
8,prep S2C2T2,0,6.5,1.0,6.5,1,0,1.0,2.0,True,"[11, 20, 20, 20, 20, 20]"
9,prep S2C2T3,0,6.5,1.0,6.5,1,1,1.0,2.0,True,"[2, 20, 20, 20, 20, 20]"


In [28]:
Controleren.EvaluateProgram(dfSprint,dfk,dfKalenderCrew)

Het aantal uur dat nog over is: 243.0
Van het totaal aantal uren: 388.0
Het aantal uur dat moest worden ingepland: 209.0
Het aantal opdrachten dat niet zijn ingepland: 0


In [29]:
dfKalenderCrew.iloc[5,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,0,0,0,0,0,0,7,1,1,0,0,1,7,7,7
1,09:00:00,S2C1T3 freq: 1 workspace B,0,S2C2T3 freq: 1 workshop B,S2C1T2 freq: 1 assembly A,0,S2C1T5A freq: 1 workspace A,S2C1T2 freq: 1 assembly A,0,S2C1T3 freq: 1 workspace B,S2C1T4 freq: 1 workspace C,S2C1T5A freq: 1 workspace A,S2C3T2A freq: 1 workshop A,S2C1T4 freq: 1 workspace C,0,0,0
2,09:30:00,S2C1T3 freq: 1 workspace B,0,S2C2T3 freq: 1 workshop B,S2C1T2 freq: 1 assembly A,0,S2C1T5A freq: 1 workspace A,S2C1T2 freq: 1 assembly A,0,S2C1T3 freq: 1 workspace B,S2C1T4 freq: 1 workspace C,S2C1T5A freq: 1 workspace A,S2C3T2A freq: 1 workshop A,S2C1T4 freq: 1 workspace C,0,0,0
3,10:00:00,S2C1T3 freq: 1 workspace B,0,S2C2T3 freq: 1 workshop B,S2C1T2 freq: 1 assembly A,0,S2C1T5A freq: 1 workspace A,S2C1T2 freq: 1 assembly A,0,S2C1T3 freq: 1 workspace B,S2C1T4 freq: 1 workspace C,S2C1T5A freq: 1 workspace A,S2C3T2A freq: 1 workshop A,S2C1T4 freq: 1 workspace C,0,0,0
4,10:30:00,S2C1T3 freq: 1 workspace B,0,S2C2T3 freq: 1 workshop B,S2C1T2 freq: 1 assembly A,0,S2C1T5A freq: 2 workspace A,S2C1T2 freq: 1 assembly A,0,S2C1T3 freq: 1 workspace B,S2C1T4 freq: 1 workspace C,S2C1T5A freq: 2 workspace A,S2C3T2A freq: 1 workshop A,S2C1T4 freq: 1 workspace C,0,0,0
5,11:00:00,S2C1T3 freq: 2 workspace B,0,S2C2T3 freq: 2 workshop B,S2C1T2 freq: 2 assembly A,0,S2C1T5A freq: 2 workspace A,S2C1T2 freq: 2 assembly A,0,S2C1T3 freq: 2 workspace B,S2C1T4 freq: 2 workspace C,S2C1T5A freq: 2 workspace A,S2C2T2 freq: 1 workshop A,S2C1T4 freq: 2 workspace C,0,0,0
6,11:30:00,S2C1T3 freq: 2 workspace B,0,S2C2T3 freq: 2 workshop B,S2C1T2 freq: 2 assembly A,0,S2C1T5A freq: 2 workspace A,S2C1T2 freq: 2 assembly A,0,S2C1T3 freq: 2 workspace B,S2C1T4 freq: 2 workspace C,S2C1T5A freq: 2 workspace A,S2C2T2 freq: 1 workshop A,S2C1T4 freq: 2 workspace C,0,0,0
7,12:30:00,S2C1T3 freq: 2 workspace B,0,S2C2T3 freq: 2 workshop B,S2C1T2 freq: 2 assembly A,0,S2C1T5A freq: 3 workspace A,S2C1T2 freq: 2 assembly A,0,S2C1T3 freq: 2 workspace B,S2C1T4 freq: 2 workspace C,S2C1T5A freq: 3 workspace A,S2C2T2 freq: 1 workshop A,S2C1T4 freq: 2 workspace C,0,0,0
8,13:00:00,S2C1T3 freq: 2 workspace B,0,S2C2T3 freq: 2 workshop B,S2C1T2 freq: 2 assembly A,0,S2C1T5A freq: 3 workspace A,S2C1T2 freq: 2 assembly A,0,S2C1T3 freq: 2 workspace B,S2C1T4 freq: 2 workspace C,S2C1T5A freq: 3 workspace A,S2C2T2 freq: 1 workshop A,S2C1T4 freq: 2 workspace C,0,0,0
9,13:30:00,S2C1T3 freq: 3 workspace B,0,S2C2T3 freq: 3 workshop B,S2C1T2 freq: 3 assembly A,0,S2C1T5A freq: 3 workspace A,S2C1T2 freq: 3 assembly A,0,S2C1T3 freq: 3 workspace B,S2C1T4 freq: 3 workspace C,S2C1T5A freq: 3 workspace A,S2C2T2 freq: 2 workshop A,S2C1T4 freq: 3 workspace C,0,0,0


In [30]:
dfKalenderCrew.iloc[6,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,0.5,1.5,1.5,0,0,1.5,2.5,0.5,0,0.5,0.5,2.5,6.5,7,7
1,09:00:00,OSHA,S2C3T4 freq: 1 workshop C,S2C2T3 freq: 4 workshop B,S2C1T2 freq: 4 assembly A,OSHA,S2C1T5A freq: 5 workspace A,S2C1T2 freq: 4 assembly A,0,S2C3T4 freq: 1 workshop C,OSHA,S2C1T5A freq: 5 workspace A,S2C2T2 freq: 3 workshop A,0,0,0,0
2,09:30:00,OSHA,S2C3T4 freq: 1 workshop C,S2C2T3 freq: 4 workshop B,S2C1T2 freq: 4 assembly A,OSHA,S2C1T5A freq: 6 workspace B,S2C1T2 freq: 4 assembly A,0,S2C3T4 freq: 1 workshop C,OSHA,S2C1T5A freq: 6 workspace B,S2C2T2 freq: 3 workshop A,0,0,0,0
3,10:00:00,OSHA,S2C3T4 freq: 1 workshop C,0,0,OSHA,S2C1T5A freq: 6 workspace B,0,0,S2C3T4 freq: 1 workshop C,OSHA,S2C1T5A freq: 6 workspace B,0,0,0,0,0
4,10:30:00,OSHA,S2C3T4 freq: 1 workshop C,0,0,OSHA,S2C1T5A freq: 6 workspace B,0,0,S2C3T4 freq: 1 workshop C,OSHA,S2C1T5A freq: 6 workspace B,0,0,0,0,0
5,11:00:00,OSHA,0,0,0,OSHA,0,0,0,0,OSHA,0,0,0,0,0,0
6,11:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,0,0,0
7,12:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,0,0,0
8,13:00:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,0,0,0
9,13:30:00,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,weekly stand-up,0,0,0


In [31]:
dfKalenderCrew.iloc[7,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,0,0,0,1,0.5,0,0,3,0,0,1,0,3.5,7,7
1,09:00:00,0,0,S2C3T3 freq: 1 workshop C,0,S2C2T4A freq: 1 assembly A,S2C2T5 freq: 1 workshop B,0,0,0,S2C1T4 freq: 4 workspace A,0,S2C2T2 freq: 4 workshop A,S2C1T4 freq: 4 workspace A,S2C2T5 freq: 1 workshop B,0,0
2,09:30:00,0,0,S2C3T3 freq: 1 workshop C,0,S2C2T4A freq: 1 assembly A,S2C2T5 freq: 1 workshop B,0,0,0,S2C1T4 freq: 4 workspace A,0,S2C2T2 freq: 4 workshop A,S2C1T4 freq: 4 workspace A,S2C2T5 freq: 1 workshop B,0,0
3,10:00:00,0,0,S2C3T3 freq: 1 workshop C,0,S2C2T4A freq: 1 assembly A,S2C2T5 freq: 1 workshop B,0,0,0,S2C1T4 freq: 4 workspace A,0,S2C3T2B freq: 1 assembly B,S2C1T4 freq: 4 workspace A,S2C2T5 freq: 1 workshop B,0,0
4,10:30:00,0,0,S2C3T3 freq: 1 workshop C,0,S2C2T4A freq: 2 assembly A,0,0,0,0,S2C1T4 freq: 4 workspace A,0,S2C3T2B freq: 1 assembly B,S2C1T4 freq: 4 workspace A,0,0,0
5,11:00:00,0,0,0,0,S2C2T4A freq: 2 assembly A,0,0,0,0,S2C1T4 freq: 5 workspace A,0,S2C3T2B freq: 1 assembly B,S2C1T4 freq: 5 workspace A,0,0,0
6,11:30:00,0,0,0,0,S2C2T4A freq: 2 assembly A,0,0,0,0,S2C1T4 freq: 5 workspace A,0,S2C3T2B freq: 1 assembly B,S2C1T4 freq: 5 workspace A,0,0,0
7,12:30:00,0,0,S2C2T4B freq: 1 technical A,0,S2C3T1 freq: 1 workshop A,S2C3T1 freq: 1 workshop A,0,0,S2C2T4B freq: 1 technical A,S2C1T4 freq: 5 workspace A,0,S2C3T1 freq: 1 workshop A,S2C1T4 freq: 5 workspace A,0,0,0
8,13:00:00,0,0,S2C2T4B freq: 1 technical A,0,S2C3T1 freq: 1 workshop A,S2C3T1 freq: 1 workshop A,0,0,S2C2T4B freq: 1 technical A,S2C1T4 freq: 5 workspace A,0,S2C3T1 freq: 1 workshop A,S2C1T4 freq: 5 workspace A,0,0,0
9,13:30:00,0,0,S2C2T4B freq: 1 technical A,0,S2C3T1 freq: 1 workshop A,S2C3T1 freq: 1 workshop A,0,0,S2C2T4B freq: 1 technical A,S2C1T4 freq: 6 workspace A,0,S2C3T1 freq: 1 workshop A,S2C1T4 freq: 6 workspace A,0,0,0


In [32]:
dfKalenderCrew.iloc[8,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,2,6,2,7,0,7,2,0,4,1,1,4,7,7,7
1,09:00:00,S2C1T3 freq: 4 workspace B,S2C2T1 freq: 1 assembly B,S2C3T3 freq: 2 workshop A,S2C2T1 freq: 1 assembly B,0,S2C1T5A freq: 7 workspace A,0,S2C2T1 freq: 1 assembly B,S2C1T3 freq: 4 workspace B,S2C1T4 freq: 7 assembly A,S2C1T5A freq: 7 workspace A,S2C2T1 freq: 1 assembly B,S2C1T4 freq: 7 assembly A,0,0,0
2,09:30:00,S2C1T3 freq: 4 workspace B,S2C2T1 freq: 1 assembly B,S2C3T3 freq: 2 workshop A,S2C2T1 freq: 1 assembly B,0,S2C1T5A freq: 7 workspace A,0,S2C2T1 freq: 1 assembly B,S2C1T3 freq: 4 workspace B,S2C1T4 freq: 7 assembly A,S2C1T5A freq: 7 workspace A,S2C2T1 freq: 1 assembly B,S2C1T4 freq: 7 assembly A,0,0,0
3,10:00:00,S2C1T3 freq: 4 workspace B,S2C2T1 freq: 1 assembly B,0,S2C2T1 freq: 1 assembly B,0,S2C1T5A freq: 7 workspace A,0,S2C2T1 freq: 1 assembly B,S2C1T3 freq: 4 workspace B,S2C1T4 freq: 8 workspace C,S2C1T5A freq: 7 workspace A,S2C2T1 freq: 1 assembly B,S2C1T4 freq: 8 workspace C,0,0,0
4,10:30:00,S2C1T3 freq: 4 workspace B,S2C2T1 freq: 1 assembly B,0,S2C2T1 freq: 1 assembly B,0,S2C1T5A freq: 8 workspace A,0,S2C2T1 freq: 1 assembly B,S2C1T3 freq: 4 workspace B,S2C1T4 freq: 8 workspace C,S2C1T5A freq: 8 workspace A,S2C2T1 freq: 1 assembly B,S2C1T4 freq: 8 workspace C,0,0,0
5,11:00:00,S2C1T3 freq: 5 workspace B,0,0,0,0,S2C1T5A freq: 8 workspace A,0,0,S2C1T3 freq: 5 workspace B,S2C1T4 freq: 8 workspace C,S2C1T5A freq: 8 workspace A,0,S2C1T4 freq: 8 workspace C,0,0,0
6,11:30:00,S2C1T3 freq: 5 workspace B,0,0,0,0,S2C1T5A freq: 8 workspace A,0,0,S2C1T3 freq: 5 workspace B,S2C1T4 freq: 8 workspace C,S2C1T5A freq: 8 workspace A,0,S2C1T4 freq: 8 workspace C,0,0,0
7,12:30:00,S2C1T3 freq: 5 workspace B,S2C1T1 freq: 1 assembly A,0,S2C1T1 freq: 1 assembly A,0,S2C1T5B freq: 1 technical A,0,S2C1T1 freq: 1 assembly A,S2C1T3 freq: 5 workspace B,0,S2C1T1 freq: 1 assembly A,S2C1T5B freq: 1 technical A,0,0,0,0
8,13:00:00,S2C1T3 freq: 5 workspace B,S2C1T1 freq: 1 assembly A,0,S2C1T1 freq: 1 assembly A,0,S2C1T5B freq: 1 technical A,0,S2C1T1 freq: 1 assembly A,S2C1T3 freq: 5 workspace B,0,S2C1T1 freq: 1 assembly A,S2C1T5B freq: 1 technical A,0,0,0,0
9,13:30:00,S2C1T3 freq: 6 workspace A,S2C1T1 freq: 1 assembly A,0,S2C1T1 freq: 1 assembly A,0,S2C1T5B freq: 1 technical A,0,S2C1T1 freq: 1 assembly A,S2C1T3 freq: 6 workspace A,0,S2C1T1 freq: 1 assembly A,S2C1T5B freq: 1 technical A,0,0,0,0


In [33]:
dfKalenderCrew.iloc[9,:].DagRooster.dfRooster

,Tijd,Alyssa,Chris,Dennis,Iris,Jurgen,Lois,Melanie,Mitch,Rudy,Shane,Wendy,William,Winston,Extra 1,Extra 2,Extra 3
0,urenTeGaan,0,0,3,0,0,3,7,7,0,0,0,3,7,7,7,7
1,09:00:00,S2C1T3 freq: 7 workspace A,0,0,0,0,S2C1T5B freq: 3 technical A,0,0,S2C1T3 freq: 7 workspace A,0,0,S2C1T5B freq: 3 technical A,0,0,0,0
2,09:30:00,S2C1T3 freq: 7 workspace A,0,0,0,0,S2C1T5B freq: 3 technical A,0,0,S2C1T3 freq: 7 workspace A,0,0,S2C1T5B freq: 3 technical A,0,0,0,0
3,10:00:00,S2C1T3 freq: 8 workspace B,0,0,0,0,S2C1T5B freq: 3 technical A,0,0,S2C1T3 freq: 8 workspace B,0,0,S2C1T5B freq: 3 technical A,0,0,0,0
4,10:30:00,S2C1T3 freq: 8 workspace B,0,0,0,0,S2C1T5B freq: 3 technical A,0,0,S2C1T3 freq: 8 workspace B,0,0,S2C1T5B freq: 3 technical A,0,0,0,0
5,11:00:00,S2C1T3 freq: 8 workspace B,0,0,0,0,S2C1T5B freq: 4 technical A,0,0,S2C1T3 freq: 8 workspace B,0,0,S2C1T5B freq: 4 technical A,0,0,0,0
6,11:30:00,S2C1T3 freq: 8 workspace B,0,0,0,0,S2C1T5B freq: 4 technical A,0,0,S2C1T3 freq: 8 workspace B,0,0,S2C1T5B freq: 4 technical A,0,0,0,0
7,12:30:00,S2C1T6 freq: 2 workspace A,0,S2C2T4B freq: 3 technical B,0,0,S2C1T5B freq: 4 technical A,0,0,S2C2T4B freq: 3 technical B,0,0,S2C1T5B freq: 4 technical A,0,0,0,0
8,13:00:00,S2C1T6 freq: 2 workspace A,0,S2C2T4B freq: 3 technical B,0,0,S2C1T5B freq: 4 technical A,0,0,S2C2T4B freq: 3 technical B,0,0,S2C1T5B freq: 4 technical A,0,0,0,0
9,13:30:00,S2C1T6 freq: 3 workspace A,0,S2C2T4B freq: 3 technical B,0,0,0,0,0,S2C2T4B freq: 3 technical B,0,0,0,0,0,0,0


In [34]:
import sys
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('dfKalenderCrew', 11904),
 ('dfKalenderRoom', 11904),
 ('dfk', 11904),
 ('dfr', 11904),
 ('dfSprintAddprep', 10900),
 ('dfSprintAddpost', 10885),
 ('dfSprint', 10852),
 ('dfs', 10602),
 ('dfprep', 9556),
 ('dfpost', 9541),
 ('dfSprintAdd', 8489),
 ('eis', 4937),
 ('task', 4894),
 ('dfCrew', 4200),
 ('dfc', 4200),
 ('dfRoosterKamer', 3816),
 ('dfRooms', 1863),
 ('MogelijkeCombinaties', 1056),
 ('Rooster', 1056),
 ('product', 400),
 ('counter', 216),
 ('rows', 216),
 ('InplannenAdd', 136),
 ('InplannenOpdracht', 136),
 ('yapf_reformat', 136),
 ('OSHA', 88),
 ('np', 80),
 ('pd', 80),
 ('crewA', 64),
 ('SPRINT', 51),
 ('crewDirecter', 28),
 ('dagIndex', 28),
 ('i', 28),
 ('iteratie', 28),
 ('j', 28),
 ('legal', 28),
 ('prepCheck', 28),
 ('prepTrouble', 28),
 ('science', 28),
 ('PR', 24)]

In [35]:
dfKalenderRoom.iloc[8,:].DagRooster.dfRooster

,Tijd,assembly A,assembly B,workshop A,workshop B,workshop C,workspace A,workspace B,workspace C,technical A,...,office A4,office A5,office B1,office B2,office B3,office B4,office C1,office C2,office C3,office C4
0,09:00:00,S2C1T4 freq: 7,S2C2T1 freq: 1,S2C3T3 freq: 2,0,0,S2C1T5A freq: 7,S2C1T3 freq: 4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,09:30:00,S2C1T4 freq: 7,S2C2T1 freq: 1,S2C3T3 freq: 2,0,0,S2C1T5A freq: 7,S2C1T3 freq: 4,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10:00:00,S2C1T4 freq: 7,S2C2T1 freq: 1,S2C3T3 freq: 2,0,0,S2C1T5A freq: 7,S2C1T3 freq: 4,S2C1T4 freq: 8,0,...,0,0,0,0,0,0,0,0,0,0
3,10:30:00,0,S2C2T1 freq: 1,0,0,0,S2C1T5A freq: 8,S2C1T3 freq: 4,S2C1T4 freq: 8,0,...,0,0,0,0,0,0,0,0,0,0
4,11:00:00,0,0,0,0,0,S2C1T5A freq: 8,S2C1T3 freq: 5,S2C1T4 freq: 8,0,...,0,0,0,0,0,0,0,0,0,0
5,11:30:00,0,0,0,0,0,S2C1T5A freq: 8,S2C1T3 freq: 5,S2C1T4 freq: 8,0,...,0,0,0,0,0,0,0,0,0,0
6,12:30:00,S2C1T1 freq: 1,0,0,0,0,0,S2C1T3 freq: 5,0,S2C1T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
7,13:00:00,S2C1T1 freq: 1,0,0,0,0,0,S2C1T3 freq: 5,0,S2C1T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
8,13:30:00,S2C1T1 freq: 1,0,0,0,0,S2C1T3 freq: 6,0,0,S2C1T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0
9,14:00:00,S2C1T1 freq: 1,0,0,0,0,S2C1T3 freq: 6,0,0,S2C1T5B freq: 1,...,0,0,0,0,0,0,0,0,0,0


!TODO Hierin kloppen de uren van de extra crew sowieso niet, wel van de rest?

In [36]:
dfCrew

,Crew members,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Skill 7,Skill 8,Skill 9,...,Skill 11,Skill 12,Skill 13,mon,tue,wed,thu,fri,Additional,Uren
0,Alyssa,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,1,1,0,39.0
1,Chris,1,0,0,0,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,18.0
2,Dennis,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,0,116.0
3,Iris,1,0,0,0,0,1,0,1,0,...,1,0,1,1,1,0,1,0,0,46.0
4,Jurgen,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,0,0,10.0
5,Lois,1,1,1,0,0,0,1,0,1,...,0,0,0,1,1,1,1,1,must have acquired skill 8 by the end of sprint 4,8.0
6,Melanie,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,6.0
7,Mitch,1,0,1,0,0,0,0,1,1,...,1,1,1,1,1,0,1,1,0,172.0
8,Rudy,1,1,0,0,0,0,1,1,1,...,1,0,0,1,1,1,1,1,0,81.0
9,Shane,1,0,0,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,36.5


In [37]:
# # Het bedrijf werkt in sprints. Om precies te zijn: 4 sprints van elk 10 weken. 
# # De weken 2 t/m 8 van elke sprint zijn de zogeheten projectweken
# # het kopieren van de roosters. Deze zijn voor dit gedeelte gelijk in week 2 tot en met 8. 


for i in range(3,9):
    for j in range(5,10):
        dfKalenderRoom.iloc[(i-2)*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderRoom.iloc[j,:].DagRooster.dfRooster))
        dfKalenderCrew.iloc[(i-2)*5 + j,:].DagRooster.dfRooster = pickle.loads(pickle.dumps(dfKalenderCrew.iloc[j,:].DagRooster.dfRooster))
    

In [38]:
# # ophalen opgeslagen bestanden zodat het eerste stuk niet hoeft te worden gerund. 
# # dit wordt alleen gebruikt bij hogere iteraties die op meerdere momenten worden gerund
# filehandler = open('dfSprint1_eind.obj', 'rb') 
# dfSprint = pickle.load(filehandler)
# filehandler = open('dfKalenderCrew1.obj', 'rb') 
# dfKalenderCrew = pickle.load(filehandler)
# filehandler = open('dfKalenderRoom1.obj', 'rb') 
# dfKalenderRoom = pickle.load(filehandler)
# filehandler = open('dfSprintAddprep1.obj', 'rb') 
# dfSprintAddprep = pickle.load(filehandler)
# filehandler = open('dfSprintAddpost1.obj', 'rb') 
# dfSprintAddpost = pickle.load(filehandler)
# filehandler = open('dfCrew1.obj', 'rb') 
# dfCrew = pickle.load(filehandler)

# filehandler.close()

In [39]:
def InplannenExtraOpdrachten(task, crew, dfKalenderCrew, dfSprintAdd, dag=0, uur=0):
    dagkopie = dag
    halveuren = 0
    task = task.reset_index(drop=True)
    while ((halveuren < int(task['DurationExp.1'][0]*2)) & (dag < 50)): 
        if (dfKalenderCrew.iloc[dag,:].DagRooster.dfRooster.iloc[0,crew+1] > 0):
            for j in range(1,15):
                if dagkopie==dag & j<=uur:
                    j=uur+1
                if (dfKalenderCrew.iloc[dag,:].DagRooster.dfRooster.iloc[j,crew+1] == 0): 
                    dfKalenderCrew.iloc[dag,:].DagRooster.dfRooster.iloc[j,crew+1] = task.Task[0]

                    dfKalenderCrew.iloc[dag,:].DagRooster.dfRooster.iloc[0,crew+1] -= 0.5
                    halveuren += 1
                    if (dfKalenderCrew.iloc[dag,:].DagRooster.dfRooster.iloc[0,crew+1] == 0):
                        break
        dag +=1
        
    dfSprintAdd.loc[dfSprintAdd['Task'] == task.Task[0], 'Voltooid'] = (halveuren == int(task['DurationExp.1'][0]*2))
                                                                     
    return [dfKalenderCrew, dfSprintAdd, dag, uur]

In [40]:
dfKalenderCrew, dfSprintAdd, dagcheck, uurcheck = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "prep check-up"], prepCheck, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dagtrouble, uurtrouble = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "prep troubleshooting"], prepTrouble, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dag, uur = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "meetup with stakeholders"], crewDirecter, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dag, uur = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "crew director"], crewDirecter, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dag, uur = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "scientific advisor"], science, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dag, uur = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "PR"], PR, dfKalenderCrew, dfSprintAdd)
dfKalenderCrew, dfSprintAdd, dag, uur = InplannenExtraOpdrachten(dfSprintAdd[dfSprintAdd.Task == "legal assistant"], legal, dfKalenderCrew, dfSprintAdd)

# check-up, troubleshooting, special trouble shooting

In [41]:
# check-up
Crew_10_11 = dfCrew[(dfCrew['Skill 10'] == 1) & (dfCrew['Skill 11'] == 1)]
Crew_10 =  dfCrew[(dfCrew['Skill 10'] == 1) & (dfCrew['Skill 11'] == 0)]

freq = 0
task1 = dfSprintAdd[dfSprintAdd.Task.str.contains("check-up 1")].reset_index(drop=True) # weet niet zeker of deze goed wordt meegegeven aan de functie
task2 = dfSprintAdd[dfSprintAdd.Task.str.contains("check-up 2")].reset_index(drop=True)
while freq < task2['aantal keer'][0]:
    for index, crew in Crew_10_11.iterrows():
        if (crew.Uren >= task2['DurationExp.1'][0]):
            if (freq == task2['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task2.Task[0], 'Voltooid'] = (freq == task1['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task2, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1
    
    for index, crew in Crew_10.iterrows():
        if (crew.Uren >= task1['DurationExp.1'][0]):
            if (freq == task1['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task2.Task[0], 'Voltooid'] = (freq == task1['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task1, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1

In [42]:
#troubleshooting
Crew_10 =  dfCrew[(dfCrew['Skill 10'] == 1)]

freq = 0
task = dfSprintAdd[dfSprintAdd.Task.str.contains("troubleshooting")].reset_index(drop=True)
while freq < task['aantal keer'][0]:
    
    for index, crew in Crew_10.iterrows():
        if (crew.Uren >= task['DurationExp.1'][0]):
            if (freq == task['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task.Task[0], 'Voltooid'] = (freq == task1['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1

In [43]:
#special troubleshooting
Crew_5_11 = dfCrew[(dfCrew['Skill 5'] == 1) & (dfCrew['Skill 11'] == 1) & (dfCrew['Skill 4'] == 0)]
Crew_4_11 = dfCrew[(dfCrew['Skill 5'] == 0) & (dfCrew['Skill 11'] == 1) & (dfCrew['Skill 4'] == 1)]
Crew_4_5_11 = dfCrew[(dfCrew['Skill 5'] == 1) & (dfCrew['Skill 11'] == 1) & (dfCrew['Skill 4'] == 1)]

freq = 0
task1 = dfSprintAdd[dfSprintAdd.Task.str.contains("special troubleshooting 1")].reset_index(drop=True) # weet niet zeker of deze goed wordt meegegeven aan de functie
task2 = dfSprintAdd[dfSprintAdd.Task.str.contains("special troubleshooting 2")].reset_index(drop=True)
task3 = dfSprintAdd[dfSprintAdd.Task.str.contains("special troubleshooting 3")].reset_index(drop=True)
while freq < task3['aantal keer'][0]:
    for index, crew in Crew_4_5_11.iterrows():
        if (crew.Uren >= task3['DurationExp.1'][0]):
            if (freq == task3['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task3.Task[0], 'Voltooid'] = (freq == task3['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task3, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1
    
    for index, crew in Crew_4_11.iterrows():
        if (crew.Uren >= task3['DurationExp.1'][0]):
            if (freq == task3['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task3.Task[0], 'Voltooid'] = (freq == task3['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task2, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1

    for index, crew in Crew_5_11.iterrows():
        if (crew.Uren >= task3['DurationExp.1'][0]):
            if (freq == task3['aantal keer'][0]):
                dfSprintAdd.loc[dfSprintAdd['Task'] == task3.Task[0], 'Voltooid'] = (freq == task3['aantal keer'][0]) 
                break
            dfKalenderCrew, dfSprintAdd = InplannenExtraOpdrachten(task1, dfCrew.index[dfCrew['Crew members'] == crew['Crew members']].tolist()[0], dfKalenderCrew, dfSprintAdd)
            freq += 1

In [44]:
dfSprintAdd

,Task,ReqSkill 1,ReqSkill 2,ReqSkill 3,ReqSkill 4,ReqSkill 5,ReqSkill 6,ReqSkill 7,ReqSkill 8,ReqSkill 9,...,DurationExp.3,DurationStd.3,People.1,People.2,People.3,Condition,aantal keer,Sprint,Voltooid,Crew
0,prep check-up,0,0.0,0.0,0,0,0.0,0,0,0,...,30.0,9.0,1,1,1,0,1.0,2.0,False,20
1,check-up 1,0,0.0,0.0,0,0,0.0,0,0,0,...,10.0,2.0,0,0,0,0,10.0,2.0,True,20
2,check-up 2,0,0.0,0.0,0,0,0.0,0,0,0,...,6.0,1.0,0,0,0,0,10.0,2.0,True,20
3,prep troubleshooting,0,0.0,0.0,0,0,0.0,0,0,0,...,75.0,25.0,0,0,0,in sprint 2,1.0,2.0,False,20
4,troubleshooting,0,0.0,0.0,0,0,0.0,0,0,0,...,1.5,0.5,1,1,1,0,1.0,2.0,False,20
5,special troubleshooting 1,0,0.0,0.0,0,1,0.0,0,0,0,...,12.0,4.0,0,0,0,0,1.0,2.0,False,20
6,special troubleshooting 2,0,0.0,0.0,1,0,0.0,0,0,0,...,8.0,3.0,0,0,0,0,1.0,2.0,False,20
7,special troubleshooting 3,0,0.0,0.0,1,1,0.0,0,0,0,...,6.0,2.0,0,0,0,0,1.0,2.0,True,20
8,OSHA committee,0,0.0,0.0,0,0,0.0,0,1,0,...,25.0,5.0,3,3,3,0,1.0,2.0,True,"[4, 9, 0]"
9,legal assistant,0,0.0,0.0,0,0,0.0,0,1,0,...,40.0,8.0,1,1,1,0,1.0,2.0,False,20


In [45]:
dfCrew

,Crew members,Skill 1,Skill 2,Skill 3,Skill 4,Skill 5,Skill 6,Skill 7,Skill 8,Skill 9,...,Skill 11,Skill 12,Skill 13,mon,tue,wed,thu,fri,Additional,Uren
0,Alyssa,0,0,0,0,0,0,0,1,0,...,0,1,0,1,1,0,1,1,0,39.0
1,Chris,1,0,0,0,0,1,1,1,1,...,0,1,0,0,1,0,1,0,0,18.0
2,Dennis,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,1,0,116.0
3,Iris,1,0,0,0,0,1,0,1,0,...,1,0,1,1,1,0,1,0,0,46.0
4,Jurgen,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,0,0,10.0
5,Lois,1,1,1,0,0,0,1,0,1,...,0,0,0,1,1,1,1,1,must have acquired skill 8 by the end of sprint 4,8.0
6,Melanie,1,0,1,0,0,0,0,1,1,...,1,0,0,1,1,0,1,1,0,6.0
7,Mitch,1,0,1,0,0,0,0,1,1,...,1,1,1,1,1,0,1,1,0,172.0
8,Rudy,1,1,0,0,0,0,1,1,1,...,1,0,0,1,1,1,1,1,0,81.0
9,Shane,1,0,0,1,0,0,0,1,1,...,0,1,1,1,1,1,1,0,0,36.5


In [46]:
# filehandler = open('dfSprint1_eind.obj', 'wb') 
# pickle.dump(dfSprint, filehandler)

# filehandler = open('dfKalenderCrew1.obj', 'wb') 
# pickle.dump(dfKalenderCrew, filehandler)

# filehandler = open('dfKalenderRoom1.obj', 'wb') 
# pickle.dump(dfKalenderRoom, filehandler)

# filehandler = open('dfSprintAddprep1.obj', 'wb') 
# pickle.dump(dfSprintAddprep, filehandler)

# filehandler = open('dfSprintAddpost1.obj', 'wb') 
# pickle.dump(dfSprintAddpost, filehandler)

# filehandler = open('dfCrew1.obj', 'wb') 
# pickle.dump(dfCrew, filehandler)

# filehandler.close()